In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img

import cv2

from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor

from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from os import makedirs
from os.path import expanduser, exists, join

from sklearn import preprocessing
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [2]:
train_dogs = pd.read_csv('labels.csv')

In [3]:
train_folder = 'train/'

In [4]:
new_data = pd.DataFrame()
for i in train_dogs['breed'].unique():
    length = len(train_dogs[train_dogs['breed'] == i])
    if length >= 90:
        new_data = new_data.append(train_dogs[train_dogs['breed'] == i])
new_data

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
201,04b5834c3a969c6308ce157c547bb313,basenji
287,06faa041b335551e3ccf3c239d006425,basenji
301,073f0821a0842917e6e97ab322bd63f2,basenji
397,0990fe6be15d9d556eac8712db3c8094,basenji
...,...,...
9669,f375e6363bc21dcd3cb65637c7855e9c,silky_terrier
9708,f42afcc1fabac064a07c6a5b7ae2766e,silky_terrier
9904,f8d48f89aaa55962d4beb853a128eac7,silky_terrier
9961,fa26802c7a0ff2fc7dcabb9999a6c4b3,silky_terrier


In [5]:
train_dogs = new_data
train_dogs

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
201,04b5834c3a969c6308ce157c547bb313,basenji
287,06faa041b335551e3ccf3c239d006425,basenji
301,073f0821a0842917e6e97ab322bd63f2,basenji
397,0990fe6be15d9d556eac8712db3c8094,basenji
...,...,...
9669,f375e6363bc21dcd3cb65637c7855e9c,silky_terrier
9708,f42afcc1fabac064a07c6a5b7ae2766e,silky_terrier
9904,f8d48f89aaa55962d4beb853a128eac7,silky_terrier
9961,fa26802c7a0ff2fc7dcabb9999a6c4b3,silky_terrier


In [6]:
encoder = preprocessing.LabelEncoder()
train_dogs['breed'] = encoder.fit_transform(train_dogs['breed'])
train_dogs

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,3
201,04b5834c3a969c6308ce157c547bb313,3
287,06faa041b335551e3ccf3c239d006425,3
301,073f0821a0842917e6e97ab322bd63f2,3
397,0990fe6be15d9d556eac8712db3c8094,3
...,...,...
9669,f375e6363bc21dcd3cb65637c7855e9c,32
9708,f42afcc1fabac064a07c6a5b7ae2766e,32
9904,f8d48f89aaa55962d4beb853a128eac7,32
9961,fa26802c7a0ff2fc7dcabb9999a6c4b3,32


In [7]:
num_train_dogs = train_dogs['breed'].nunique()
num_train_dogs

35

In [8]:
target_labels = train_dogs['breed']
target_labels

8         3
201       3
287       3
301       3
397       3
         ..
9669     32
9708     32
9904     32
9961     32
10133    32
Name: breed, Length: 3587, dtype: int32

In [9]:
one_hot = pd.get_dummies(target_labels, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [10]:
train_dogs['image_path'] = train_dogs.apply( lambda x: (train_folder + x["id"] + ".jpg" ), axis=1)
train_dogs.head()

,id,breed,image_path
8,003df8b8a8b05244b1d920bb6cf451f9,3,train/003df8b8a8b05244b1d920bb6cf451f9.jpg
201,04b5834c3a969c6308ce157c547bb313,3,train/04b5834c3a969c6308ce157c547bb313.jpg
287,06faa041b335551e3ccf3c239d006425,3,train/06faa041b335551e3ccf3c239d006425.jpg
301,073f0821a0842917e6e97ab322bd63f2,3,train/073f0821a0842917e6e97ab322bd63f2.jpg
397,0990fe6be15d9d556eac8712db3c8094,3,train/0990fe6be15d9d556eac8712db3c8094.jpg


In [11]:
train_data = np.array([img_to_array(load_img(img, target_size=(299, 299))) for img in train_dogs['image_path'].values.tolist()]).astype('float32')

In [12]:
x_train, x_validation, y_train, y_validation = train_test_split(train_data, target_labels, test_size=0.2, stratify=np.array(target_labels), random_state=100)

C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, stratify=np.array(y_train), random_state=100)

C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

In [14]:
print ('x_train shape = ', x_train.shape)
print ('x_validation shape = ', x_validation.shape)
print ('x_test shape = ', x_test.shape)

x_train shape =  (2295, 299, 299, 3)
x_validation shape =  (718, 299, 299, 3)
x_test shape =  (574, 299, 299, 3)


In [15]:
y_train_cold = y_train
y_validation_cold = y_validation
y_test_cold = y_test

In [16]:
y_train = pd.get_dummies(y_train.reset_index(drop=True)).as_matrix()
y_validation = pd.get_dummies(y_validation.reset_index(drop=True)).as_matrix()
y_test = pd.get_dummies(y_test.reset_index(drop=True)).as_matrix()

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   # zoom_range = 0.3, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')
train_generator = train_datagen.flow(x_train, y_train, shuffle=False, batch_size=10, seed=10)

In [18]:
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow(x_validation, y_validation, shuffle=False, batch_size=10, seed=10)

In [19]:
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(299, 299, 3))

In [20]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [21]:
x = Dense(512, activation='relu')(x)
predictions = Dense(num_train_dogs, activation='softmax')(x)

In [22]:
model = Model(inputs = base_model.input, outputs = predictions)

In [23]:
for layer in base_model.layers:
    layer.trainable = False

In [24]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit_generator(train_generator,
                      steps_per_epoch = 175,
                      validation_data = val_generator,
                      validation_steps = 44,
                      epochs = 10,
                      verbose = 2)

Epoch 1/10
175/175 - 161s - loss: 1.9212 - accuracy: 0.6499 - val_loss: 0.7622 - val_accuracy: 0.8614 - 161s/epoch - 920ms/step
Epoch 2/10
175/175 - 165s - loss: 0.4928 - accuracy: 0.8940 - val_loss: 0.3989 - val_accuracy: 0.8932 - 165s/epoch - 943ms/step
Epoch 3/10
175/175 - 148s - loss: 0.3370 - accuracy: 0.9054 - val_loss: 0.3577 - val_accuracy: 0.8886 - 148s/epoch - 844ms/step
Epoch 4/10
175/175 - 148s - loss: 0.2724 - accuracy: 0.9221 - val_loss: 0.2923 - val_accuracy: 0.9273 - 148s/epoch - 843ms/step
Epoch 5/10
175/175 - 148s - loss: 0.2619 - accuracy: 0.9203 - val_loss: 0.2880 - val_accuracy: 0.9045 - 148s/epoch - 845ms/step
Epoch 6/10
175/175 - 149s - loss: 0.2296 - accuracy: 0.9289 - val_loss: 0.2860 - val_accuracy: 0.9045 - 149s/epoch - 853ms/step
Epoch 7/10
175/175 - 147s - loss: 0.2149 - accuracy: 0.9352 - val_loss: 0.2796 - val_accuracy: 0.9000 - 147s/epoch - 841ms/step
Epoch 8/10
175/175 - 146s - loss: 0.1949 - accuracy: 0.9444 - val_loss: 0.2793 - val_accuracy: 0.9227 - 

In [26]:
predictions=model.predict(x_test)
probs = np.argmax(predictions, axis=1)
num_correct = (probs == y_test_cold).sum()
accuracy = num_correct/len(y_test_cold)
print('{:.4f}'.format(accuracy.item()))

18/18 [==============================] - 37s 2s/step
0.0348
